In [1]:
import tensorflow as tf
import keras
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import sklearn
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix

import warnings
warnings.filterwarnings('ignore')


tfk = tf.layers
tfkl = tf.layers
print("Using TensorFlow version", tf.__version__)

seed = 420
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
print("RNG seed: ",seed)

Using TensorFlow backend.


('Using TensorFlow version', '1.8.0')
('RNG seed: ', 420)


**DATA LOADING**

In [2]:
dataframe = pd.read_csv('/home/npasini1/Desktop/Recordings/nic_suture_10.csv')
# dataframe.info

In [3]:
# one hot encoding of Gestures
from numpy import array, argmax
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

gesture = dataframe['Gesture']
values = array(gesture)
print(values)

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)

encoded = to_categorical(integer_encoded)
print(encoded)

['BA' 'BA' 'BA' ... 'BA' 'BA' 'BA']
[0 0 0 ... 0 0 0]
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]


In [4]:
print(integer_encoded[793])
print(values[793])

3
TB


In [5]:
# Data normalization

labels = encoded
df = dataframe.drop(['Gesture','Timestamps'], axis=1)

import pandas as pd
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
df_scaled = min_max_scaler.fit_transform(df)
df_scaled = pd.DataFrame(df_scaled)
df_scaled.head

<bound method DataFrame.head of             0         1         2         3         4         5         6   \
0     0.099247  0.015124  0.445813  0.936466  0.140097  0.276233  0.493245   
1     0.099247  0.015124  0.445813  0.936466  0.140097  0.276233  0.493245   
2     0.099247  0.015124  0.445813  0.936466  0.140097  0.276233  0.493245   
3     0.099247  0.015124  0.445813  0.936466  0.140097  0.276233  0.493245   
4     0.099247  0.015124  0.445813  0.936466  0.140097  0.276233  0.493245   
5     0.099247  0.015124  0.445813  0.936466  0.140097  0.276233  0.493245   
6     0.099253  0.015128  0.445802  0.936452  0.140051  0.276307  0.493266   
7     0.099259  0.015132  0.445792  0.936437  0.140006  0.276380  0.493286   
8     0.099259  0.015132  0.445792  0.936437  0.140006  0.276380  0.493286   
9     0.099259  0.015132  0.445792  0.936437  0.140006  0.276380  0.493286   
10    0.099259  0.015132  0.445792  0.936437  0.140006  0.276380  0.493286   
11    0.099259  0.015132  0.4457

In [6]:
print(df_scaled.shape)

(6337, 33)


In [7]:
samples = df_scaled.shape[0]
n_features = df_scaled.shape[1]
timestep = 5
stride = 1

dataset = np.empty((0,timestep,n_features))
# np_scaled = np.array(df_scaled)
# print(dataset.shape)

# CON PREPROCESSING DEI DATI: MINMAX SCALER

# for idx in range(samples-timestep):
#     dataset = np.append(dataset, np.expand_dims(df_scaled[idx:idx+timestep], axis=0),axis=0)
#     # print(idx)

# SENZA PREPROCESSING DEI DATI

for idx in range(samples-timestep):
    dataset = np.append(dataset, np.expand_dims(df[idx:idx+timestep], axis=0),axis=0)

print(dataset.shape)
dataset_labels = labels[timestep:]

(6332, 5, 33)


In [8]:
dataset_labels = labels[timestep:]
print(dataset_labels.shape)

(6332, 4)


In [9]:
X_train, X_test, y_train, y_test = train_test_split(dataset, dataset_labels, test_size=0.10, shuffle = True)
print("Train shape: ", X_train.shape)
print("Test shape: ", X_test.shape)

input_shape = X_train.shape[1:]
output_shape = y_train.shape[1:]
print("Input shape: ", input_shape)
print("Output shape: ", output_shape)

('Train shape: ', (5698, 5, 33))
('Test shape: ', (634, 5, 33))
('Input shape: ', (5, 33))
('Output shape: ', (4,))


In [10]:
import tensorflow as tf
import keras as tfk
import keras.layers as tfkl
import scipy.io
import os, glob, sys, pickle, time, math, gc
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Model, load_model
from keras import optimizers
from keras.utils import plot_model
from keras import layers #Dense, LSTM, RepeatVector, TimeDistributed, Dropout, Masking, BatchNormalization, Flatten, Input, Conv2D, MaxPooling1D, Conv1D, Reshape, GRU
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping, CSVLogger
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
from keras import backend as K
from datetime import datetime

def buildModelv2(timesteps, n_features):
    """
    An lstm-encoder model followed by dense layers, similar performance to just lstm
    """
    # lstm_hidden1 = 128 #*4
    # lstm_hidden2 = 64
    # dense_hidden1 = 64
    # output_layer = 15

    model_input = layers.Input(shape=input_shape)
    lstm_output = layers.LSTM(128, input_shape=input_shape, kernel_regularizer=keras.regularizers.l1(l=0.001), return_sequences=True)(model_input) #previously 96
    dropout_output = layers.Dropout(rate=0.2)(lstm_output) #previously 0.4
    batch_norm1 = layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True)(dropout_output)
    lstm_output2 = layers.LSTM(64, input_shape=input_shape, kernel_regularizer=keras.regularizers.l1(l=0.001), return_sequences=False)(batch_norm1)
    dropout_output = layers.Dropout(rate = 0.3)(lstm_output2)
    batch_norm2 = layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True)(dropout_output)
    repeat_vector = layers.RepeatVector(timestep)(batch_norm2)

    flatten_output = layers.Flatten()(repeat_vector)
    batch_norm2 = layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True)(flatten_output)
    dropout_output = layers.Dropout(rate = 0.2)(batch_norm2)
    dense_output1 = layers.Dense(64, activation='relu')(dropout_output)
    batch_norm3 = layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True)(dense_output1)
    dense_output2 = layers.Dense(4, activation='softmax')(batch_norm3)
    lstm_classifier = Model(inputs=model_input, outputs=dense_output2)
    
    #Compile the model
    lstm_classifier.compile(loss=tfk.losses.categorical_crossentropy, optimizer=tfk.optimizers.Adam(1e-3), metrics=[keras.metrics.categorical_accuracy])

    return lstm_classifier

def buildModelv1(timesteps, n_features):
    """
    Simple model which yields a high accuracy of 90% on training data but sucks on the validation data
    """
    model_input = layers.Input(shape=input_shape)
    lstm_output = layers.LSTM(128, input_shape=input_shape, kernel_regularizer=keras.regularizers.l1(l=0.001), return_sequences=True)(model_input)
    dropout_output = layers.Dropout(rate=0.2)(lstm_output)
    flatten_output = layers.Flatten()(dropout_output)
    dense_output1 = layers.Dense(64, activation='relu')(flatten_output)
    dropout_output2 = layers.Dropout(rate=0.2)(dense_output1)
    dense_output2 = layers.Dense(4, activation='softmax')(dropout_output2)
    lstm_classifier = Model(model_input, dense_output2)
    #lstm_classifier.summary()
    lstm_classifier.compile(loss=tfk.losses.categorical_crossentropy, optimizer=tfk.optimizers.Adam(1e-3))

    return lstm_classifier

model = buildModelv2(timestep, n_features)
model.summary()


Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 5, 33)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 5, 128)            82944     
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 128)            0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 5, 128)            512       
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_______________________________________________________

In [11]:
mc = keras.callbacks.ModelCheckpoint('prova.h5',
                                        monitor='val_loss',
                                        mode = 'min',
                                        save_best_only=True,
                                        verbose=1)

In [13]:
batch_size = 64

history = model.fit(
    x = X_train,
    y = y_train,
    batch_size = batch_size,
    epochs = 1000,
    validation_split=.10,
    #validation_steps=10,
    shuffle=False,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=200),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=2, factor=0.5, min_lr=1e-5),
        mc
    ]
).history

Train on 5128 samples, validate on 570 samples
Epoch 1/1000
5128/5128 [==============================] - 1s - loss: 0.5744 - categorical_accuracy: 0.9503 - val_loss: 0.5565 - val_categorical_accuracy: 0.9421
Epoch 2/1000
5128/5128 [==============================] - 1s - loss: 0.4949 - categorical_accuracy: 0.9555 - val_loss: 0.4549 - val_categorical_accuracy: 0.9526
Epoch 3/1000
5128/5128 [==============================] - 1s - loss: 0.4336 - categorical_accuracy: 0.9590 - val_loss: 0.4264 - val_categorical_accuracy: 0.9474
Epoch 4/1000
5128/5128 [==============================] - 1s - loss: 0.3950 - categorical_accuracy: 0.9612 - val_loss: 0.4051 - val_categorical_accuracy: 0.9667
Epoch 5/1000
5128/5128 [==============================] - 1s - loss: 0.3430 - categorical_accuracy: 0.9700 - val_loss: 0.3613 - val_categorical_accuracy: 0.9614
Epoch 6/1000
5128/5128 [==============================] - 1s - loss: 0.3321 - categorical_accuracy: 0.9653 - val_loss: 0.3418 - val_categorical_accu

KeyboardInterrupt: 

In [14]:
model.save('mymodel.h5')

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
model = keras.models.load_model('/home/npasini1/nicolo_ws/src/pyrecorder/src/bestmodel_v2_senzapreprocessing.h5')

In [ ]:
a = model.predict(X_test)
print(X_test.shape)
prediction = np.argmax(a, axis=1)
print(prediction)